In [2]:
import os
import pandas as pd

path = "./data/cleaned_images/"
im_files = os.listdir(path)

df = pd.read_csv("./data/Images.csv")
ids = df["id"]
ids = ids.tolist()

not_here = []

for i in im_files:
    im_id = i[:-4]
    if im_id in ids:
        pass
    else:
        not_here.append(im_id)

print(len(not_here))

64


In [9]:
df1 = pd.read_csv('./data/Images.csv', delimiter=',')[['id', 'product_id']]
df2 = pd.read_csv('./data/Products.csv', lineterminator="\n")[['id', 'category']]
df3 = pd.read_csv('./data/Labels.csv').rename(columns={'Unnamed: 0' : 'idx'}).to_dict()
encoder_dict = dict((v,k) for k,v in df3['Category'].items())

df4 = pd.merge(df1, df2, left_on='product_id', right_on='id').drop(['id_y'], axis=1).rename(columns={'id_x': 'id'})
for category in df4['category']:
    main_category = category.split("/")[0].rstrip()
    df4 = df4.replace(category, main_category)

df4 = df4.replace({'category': encoder_dict})
    
df4

,id,product_id,category
0,912bb259-3ad9-457b-9db1-ce1da9016057,5f5f57d7-778f-4336-bb10-b43863418c8c,0
1,b166d305-b852-4bdd-83f4-465b20da94fa,5f5f57d7-778f-4336-bb10-b43863418c8c,0
2,68f5a29d-0075-4d60-81c1-ab684a82e50c,c2c8949f-3cde-4651-a234-4a4a1b2a9ad4,0
3,f6a309d7-d247-446a-9b5e-aceefdd4334d,c2c8949f-3cde-4651-a234-4a4a1b2a9ad4,0
4,2c2b3a6f-15b3-4289-937a-15482d9f5781,8292aa4e-7f1b-4655-bf0e-f1f2c9e3ffaf,0
...,...,...,...
12599,cdec1c5c-c4b1-42db-afbe-3fa68ea4b87d,2b0a652b-46a2-4297-b619-5efeeb222787,12
12600,dc99e40f-6b15-494d-9fb7-f0d02e9781f9,719fd40a-870e-4144-b324-55dff2e66fb4,12
12601,c8488028-bf07-4258-a4c2-56d2fe387835,719fd40a-870e-4144-b324-55dff2e66fb4,12
12602,c6113145-89c8-47cd-9211-38f29d016cc7,86d1806b-5575-4a7e-9160-f24f12be6c95,12


In [7]:
import os
import pandas as pd
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader

class Rando(Dataset):
    def __init__(self):
        super().__init__()
        self.img_dir = './data/cleaned_images/'
        self.img_labels = pd.read_csv('./data/LabelledImages.csv')
        self.transform = transforms.PILToTensor()
        
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 1] + '.jpg')
        image = Image.open(img_path)
        image = self.transform(image)
        label = self.img_labels.iloc[idx, -1]
        return image, label
    
    def __len__(self):
        return len(self.img_labels)
        
data = Rando()
loader = DataLoader(data, 1, True)
for batch in loader:
    x, y = batch
    print(x.shape), print(y.shape)
    break

torch.Size([1, 3, 512, 512])
torch.Size([1])


In [10]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

mma = torch.cuda.max_memory_allocated(device=device)
print(mma)

0


In [19]:
from datetime import datetime

timestamp = datetime.now().strftime('%d_%b_%Y_%H_%M_%S_%f')
timestamp2 = datetime.now().strftime('%d_%b_%Y_%H_%M_%S_%f')

print(timestamp)
print(timestamp2)
print(type(timestamp))

# os.makedirs(f'./model_evaluation/{timestamp}/weights', exist_ok=True)

24_Dec_2022_01_45_16_907606
24_Dec_2022_01_45_16_907637
<class 'str'>


In [24]:
# train_loss = 34.435345
# accuracy = 99.23423
# with open(f'loss_and_acc.txt', 'w') as f:
#     f.writelines(['Train Loss: %.3f'%train_loss, '\nAccuracy: %.3f'%accuracy])

In [4]:
try:
    mode = int(input('Please set the mode: '))
    print('\n1 | Train\n2 | Validation\n3 | Test')
    if mode not in (1,2,3):
        raise ValueError
except ValueError:
    print('Sorry, that is not a valid option. Please try again')


1 | Train
2 | Validation
3 | Test
You have chosen  3


In [16]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
props = torch.cuda.get_device_properties(device=device)
print(torch.cuda.get_device_name(device=device))
print(props)
print(torch.cuda.max_memory_allocated(device=device))
print(torch.cuda.max_memory_reserved(device=device))

NVIDIA GeForce RTX 2070
_CudaDeviceProperties(name='NVIDIA GeForce RTX 2070', major=7, minor=5, total_memory=7971MB, multi_processor_count=36)
0
0


In [1]:
import torch
resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)

counter = 0

for counter, child in enumerate(resnet50.children()):
    print(counter)

Using cache found in /home/diasfrancisco/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/home/diasfrancisco/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/home/diasfrancisco/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(


0
1
2
3
4
5
6


In [ ]:
list_img = sorted(os.listdir(path='./data/cleaned_images/'))
list_img

In [7]:
import torch
dict = torch.load('image_embeddings.json')
type(dict)

dict

In [6]:
import pandas as pd
labels_csv = pd.read_csv('./data/Labels.csv').rename(columns={'Unnamed: 0' : 'idx'}).to_dict()
encoder_dict = dict((v,k) for k,v in labels_csv['Category'].items())
encoder_dict['Appliances']
# type(encoder_dict['Appliances'])

9

In [14]:
import torch
from image_cnn import FtCNN
from image_processor import img_loader

model = FtCNN()
checkpoint = torch.load('./final_models/ft_model.pt')
model.load_state_dict(checkpoint)
model.eval()

with open('data/images/0a5dac34-506c-4bbd-bb6f-1c4b450942ad.jpg', "rb") as image:
    f = image.read()
    b = bytearray(f)

img = img_loader(b)
fts = model(img)
emb = fts.detach().numpy()
emb.dtype

Using cache found in /home/diasfrancisco/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


dtype('float32')

In [15]:
import torch
# '912bb259-3ad9-457b-9db1-ce1da9016057'
emb = torch.load('image_embeddings.json')
# emb_val = emb['912bb259-3ad9-457b-9db1-ce1da9016057']
# emb_val[0]
type(emb)

dict